In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.misc
from tqdm import tqdm
import imageio

# Loading/treating dataset of approx 100k images

In [2]:
# Use False if you don't want to show the daframes 
showing_data = True
#showing_data = False

In [3]:
df = pd.read_csv('../Data/faces_fakes_or_not/metadata.csv')
if showing_data:
    print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '../Data/faces_fakes_or_not/metadata.csv'

In [11]:
from tensorflow.keras.utils import image_dataset_from_directory

data_dir= '../Data/faces_fakes_or_not/faces/'

train_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(112, 112),
  batch_size=32)

val_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(112, 112),
  batch_size=16)

Found 10000 files belonging to 2 classes.
Using 8000 files for training.
Found 10000 files belonging to 2 classes.
Using 2000 files for validation.


In [12]:
from tensorflow.data import AUTOTUNE
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.metrics import Recall
from tensorflow.keras.metrics import Precision
from tensorflow.keras import regularizers

In [13]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [18]:
def initialize_model():
    reg_l1 = regularizers.L1(0.01)
    reg_l2 = regularizers.L2(0.01)
    reg_l1_l2 = regularizers.l1_l2(l1=0.005, l2=0.0005)
    model = models.Sequential()
    ## Normalization
    model.add(layers.Rescaling(1./255))
    model.build((16, 112,112,3))
    ### First convolution & max-pooling
    model.add(layers.Conv2D(64, (3,3), padding='same', activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    model.add(layers.Conv2D(64, (3,3), padding='same', activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    #model.add(layers.Conv2D(96, (3,3), padding='valid', activation="relu"))
    #model.add(layers.Conv2D(96, (3,3), padding='same', activation="relu"))
    #model.add(layers.Conv2D(48, (2,2), padding='same', activation="relu"))
    #model.add(layers.Conv2D(48, (2,2), padding='valid', activation="relu"))
    #model.add(layers.Conv2D(48, (2,2), padding='same', activation="relu"))
    #model.add(layers.MaxPool2D(pool_size=(2,2))) 
    #model.add(layers.Conv2D(24, (2,2), activation="relu"))
    #model.add(layers.MaxPool2D(pool_size=(2,2))) 
    ### Flattening
    model.add(layers.Flatten())
    ### fully connected
    #model.add(layers.Dense(64, activation='relu',activity_regularizer=reg_l1_l2))
    #model.add(layers.Dropout(rate=0.2))
    #model.add(layers.Dense(32, activation='relu',activity_regularizer=reg_l1_l2))
    #model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(30, activation='relu',activity_regularizer=reg_l1_l2))
    model.add(layers.Dropout(rate=0.2))
    ### Last layer (let's say a classification with 1 output)
    model.add(layers.Dense(1, activation='sigmoid')) 
    return model

In [19]:
def compile_model(model):
    ### Model compilation
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy',Recall(),Precision()])
    return model

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=3, restore_best_weights=True)
model = initialize_model()
model = compile_model(model)

In [21]:
history = model.fit(train_ds,
        validation_data=val_ds,
        epochs=5,
        verbose=1,
        callbacks = [es])

Epoch 1/5


2022-03-01 18:05:01.036552: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


250/250 [==============================] - 200s 793ms/step - loss: 0.6948 - accuracy: 0.5034 - recall_3: 0.7200 - precision_3: 0.5032 - val_loss: 0.6932 - val_accuracy: 0.4950 - val_recall_3: 1.0000 - val_precision_3: 0.4950
Epoch 2/5
250/250 [==============================] - 223s 891ms/step - loss: 0.6932 - accuracy: 0.4958 - recall_3: 0.6274 - precision_3: 0.4976 - val_loss: 0.6932 - val_accuracy: 0.4950 - val_recall_3: 1.0000 - val_precision_3: 0.4950
Epoch 3/5
250/250 [==============================] - 232s 928ms/step - loss: 0.6932 - accuracy: 0.4988 - recall_3: 0.7940 - precision_3: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.4950 - val_recall_3: 1.0000 - val_precision_3: 0.4950
Epoch 4/5
250/250 [==============================] - 214s 854ms/step - loss: 0.6932 - accuracy: 0.4967 - recall_3: 0.8120 - precision_3: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.4950 - val_recall_3: 1.0000 - val_precision_3: 0.4950


In [36]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

2022-03-01 15:04:03.207689: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.


(32, 224, 224, 3)
(32,)
